In [6]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import joblib

features = joblib.load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50.joblib")
label = joblib.load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib")

Esto esta resuelto en mlpAnalysis2, voy a tomar el codigo

Voy a dividir los datos a mano de la siguiente forma. Un sujeto repite cada gesto entre 5 y 6 veces. Voy a usar:

Una repetición para test.
Una para validacion.
Las restantes para train.
En este sentido, voy a guardar los indices de label en listas distintas, para identificar luego que filas de FEATURES tengo que usar para cada conjunto de datos (train, val y test).

La implementación de esta idea parte del supuesto de que los gestos iguales estan agrupados, es decir, las repeticiones del mismo gesto son consecutivas. Esto para los gestos que no son reposo.

Para los gestos de reposo se va a hacer un trato diferenciado. Primero se va a hacer la partición sin ellos, y luego, cada repetición de reposo se va a incluir en el mismo conjunto de la repetición de gesto previa a ese reposo.

In [10]:
label = np.array(label)

# Esto funciona porque los gestos iguales estan agrupados
# Hago un diff en el id del gesto para saber en que indices se cambia el gesto
# Calculo los indices de label que son ventanas de reposo para trabajar con ellas por separado
ind_reposo = np.where(label[:, 1]==0)[0]

# Calculo lo contrario ahora para facilitar obtener los labels de las ventanas con gestos
ind_gestos = np.where(label[:, 1]!=0)[0]

label_gestos = label[ind_gestos, :]

division_gestos = np.where(np.diff(label_gestos[:, 1])!=0)[0]

# Ahora hago un diff sobre la id de ventana para saber cuando cambia. Esto me va a servir para poder separarlo
# en train, val y test
division_id_ventanas = np.where(np.diff(label_gestos[:, 0])!=0)[0]

# Aca voy a guardar el id de la ventana
train = []
val = []
test = []

for i in range(len(division_gestos)+1):  #
    if i==len(division_gestos):
        ind_tope_ventana = np.max(label_gestos[:, 0])
        ind_inicio_ventana = label_gestos[division_gestos[-1], 0]

        # cantidad de repeticiones en el gesto actual (del sujeto actual)
        cant_ventanas = ind_tope_ventana - ind_inicio_ventana

    else:
        tope = division_gestos[i]   # este es limite con el proximo gesto

        if i==0:
            inicio = 0
            ind_inicio_ventana = 0
        else:
            inicio = division_gestos[i-1]
            ind_inicio_ventana = np.where(division_id_ventanas==inicio)[0][0]

        # busco en division_id_ventana cuantos elementos hay entre inicio y tope
        ind_tope_ventana = np.where(division_id_ventanas==tope)[0][0]

        # cantidad de repeticiones en el gesto actual (del sujeto actual)
        cant_ventanas = ind_tope_ventana - ind_inicio_ventana


    if i != 0:
        # sorteo las ventanas que van a cada grupo
        sorteo = np.random.permutation(cant_ventanas)
        sorteo += 1     # correccion de rango
    else:
        sorteo = np.random.permutation(cant_ventanas+1)
    # El primero va a test, el segundo a val y el resto a train
    test.append(ind_inicio_ventana + sorteo[0])
    val.append(ind_inicio_ventana + sorteo[1])
    for j in range(2, len(sorteo)):
        train.append(ind_inicio_ventana + sorteo[j])

A continuación se inserta un código de visualización. Fue usado para chequear que se estaba trabajando correctamente. Quizás pueda ser de utilidad en algún momento.

In [11]:
# Quiero visualizar con diferentes colores las ventanas que van al train, val y test
ventanas_train = []
ventanas_val = []
ventanas_test = []

# plt.figure()
# plt.plot(label_gestos[:, 1]/np.max(label_gestos[:, 1]), label = 'gesto')
# plt.plot(label_gestos[:, 0]/np.max(label_gestos[:, 0]), label = 'ventana')
# for i in range(len(train)): #
#     ventana_act_train = np.where(label_gestos[:, 0]==train[i])[0]
#     ventanas_train.append(ventana_act_train)
#     plt.plot(ventana_act_train, label_gestos[ventana_act_train, 0]/np.max(label_gestos[:, 0]), color = 'green')

#     if i < len(val):   # val y test tienen menos datos
#         ventana_act_val = np.where(label_gestos[:, 0]==val[i])[0]
#         ventanas_val.append(ventana_act_val)
#         plt.plot(ventana_act_val, label_gestos[ventana_act_val, 0]/np.max(label_gestos[:, 0]), color = 'blue')

#         ventana_act_test = np.where(label_gestos[:, 0]==test[i])[0]
#         ventanas_test.append(ventana_act_test)
#         plt.plot(ventana_act_test, label_gestos[ventana_act_test, 0]/np.max(label_gestos[:, 0]), color = 'black')

# plt.legend()

Procedo a armar los conjuntos de train, val y test para las ventanas con gesto. Tengo identificada que repeticion va para cada conjunto, ahora identifico los indices de las features usando label.

In [12]:
# Procedo a armar los conjuntos de train, val y test para las ventanas con gesto
# Tengo identificada que repeticion va para cada conjunto, ahora identifico los indices de las features usando label
indices_train = []
for j in range(len(train)):
    indices_train.extend(np.where(label[:, 0]==train[j])[0])

indices_val = []
indices_test = []
for j in range(len(val)):
    indices_val.extend(np.where(label[:, 0]==val[j])[0])
    indices_test.extend(np.where(label[:, 0]==test[j])[0])

In [13]:
# Ahora agrego las ventanas de reposo

# Voy a fijarme a que conjunto fue la repeticion anterior (que es de un gesto que no es reposo), y mandar la repeticion de reposo para ese conjunto.

# Al igual que para los otros gestos, quiero mandar todas las features de una misma "repeticion" de reposo para el mismo conjunto
div_repe_reposo = np.where(np.diff(ind_reposo)!=1)[0]
div_repe_reposo = np.insert(div_repe_reposo, 0, -1)   # el -1 es porque todos me apuntan al ultimo elemento de cada repeticion, en el for le voy a sumar 1 a todos para que me apunten al inicio que es lo que me interesa

# Calculo cantidad de repeticiones de reposo
cant_repeticiones_reposo = len(div_repe_reposo)

pertenencia_conjuntos = []  # train, val, test. Cada dato solo puede pertenecer a 1 de estos conjuntos

for i in range(len(div_repe_reposo)):
    ind0_repe_reposo_act = ind_reposo[div_repe_reposo[i]+1]
    id_repe_act = label[ind0_repe_reposo_act, 0]
    id_repe_ant = label[ind0_repe_reposo_act-1, 0]

    # id de repeticio de reposo y a que conjunto mandarlo
    pertenencia_conjuntos.append([id_repe_act, id_repe_ant in train, id_repe_ant in val, id_repe_ant in test])

pertenencia_conjuntos = np.array(pertenencia_conjuntos)

id_reposo_train = pertenencia_conjuntos[np.where(pertenencia_conjuntos[:, 1])[0], 0]
id_reposo_val = pertenencia_conjuntos[np.where(pertenencia_conjuntos[:, 2])[0], 0]
id_reposo_test = pertenencia_conjuntos[np.where(pertenencia_conjuntos[:, 3])[0], 0]

for j in range(len(id_reposo_train)):
    indices_train.extend(np.where(label[:, 0]==id_reposo_train[j])[0])

for j in range(len(id_reposo_val)):
    indices_val.extend(np.where(label[:, 0]==id_reposo_val[j])[0])
    indices_test.extend(np.where(label[:, 0]==id_reposo_test[j])[0])

%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"

joblib.dump(indices_train, 'indices_train_W200_I50.pkl')
joblib.dump(indices_val, 'indices_val_W200_I50.pkl')
joblib.dump(indices_test, 'indices_test_W200_I50.pkl')

/content/drive/My Drive/repo_tesis/archivos_generados_codigos


['indices_test_W200_I50.pkl']